In [2]:
import pandas as pd
import json
from shapely.geometry import Point, shape


/tmp/ipykernel_60178/1324844591.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Load data

In [3]:
some_datatypes = {
    "submitted_photo":"object",
    "closed_photo":"object",
    "location_zipcode":"object",
}
df = pd.read_csv('../Datasets/animal_cases.csv', dtype=some_datatypes)

with open('../Datasets/ZIP_Codes.geojson') as f:
    geojson = json.load(f)

# with pd.option_context('display.max_columns', None, 'display.width', None):
#     print(df.head())

/tmp/ipykernel_60178/2400812397.py:6: DtypeWarning: Columns (16,18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Datasets/animal_cases.csv', dtype=some_datatypes)


# Preprocessing

In [49]:
# pad the zipcode with leading zeros
df['location_zipcode'] = df['location_zipcode'].fillna(0)
df['location_zipcode'] = df['location_zipcode'].astype(int).astype(str)
df['location_zipcode'] = df['location_zipcode'].apply(lambda x: x.zfill(5))

# Fill in the missing zipcodes by longitude and latitude
def find_zipcode_if_missing(row):
    # Check if the zipcode is already present (not empty or NaN)
    if row['location_zipcode'] == "00000":
        lat, lon = row['latitude'], row['longitude']
        point = Point(lon, lat)
        for feature in geojson['features']:
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                return feature['properties']['ZIP5']
    # Return the original zipcode if it's already there, or NaN if not found
    return row['location_zipcode']

# Apply the modified function to each row
df['location_zipcode'] = df.apply(find_zipcode_if_missing, axis=1)
# save the csv back to the file
df.to_csv('../Datasets/animal_cases.csv', index=False)
with pd.option_context('display.max_columns', None, 'display.width', None):

    print(df[df['location_zipcode'] == "00000"])
df['location_zipcode'].value_counts()

## Combine the animal columns

In [58]:
animal_df = pd.read_csv('../Datasets/req_with_animals.csv')
df['animal']=animal_df['animal']
# save the csv back to the file
df.to_csv('../Datasets/animal_cases.csv', index=False)

/tmp/ipykernel_3898/302175234.py:1: DtypeWarning: Columns (13,16,18,21) have mixed types. Specify dtype option on import or set low_memory=False.
  animal_df = pd.read_csv('../Datasets/req_with_animals.csv')


## Get the Top 10 most common animals per zipcode

In [4]:
# Get the most commmon animal by zipcode
animal_by_zipcode = df.groupby(['location_zipcode', 'animal']).size().reset_index(name='animals_by_zipcode')

In [9]:
most_common_animals = df.groupby('location_zipcode')['animal'].agg(lambda x: x.value_counts().idxmax())
most_common_animals.head(n=100)

location_zipcode
00000         Cats
02108         Cats
02109         Cats
02110         Cats
02111         Cats
02113         Cats
02114         Cats
02115         Cats
02116         Cats
02118         Cats
02119         Cats
02120         Cats
02121         Cats
02122         Cats
02124         Cats
02125         Cats
02126         Cats
02127         Cats
02128         Cats
02129         Cats
02130         Cats
02131         Cats
02132         Cats
02133    Squirells
02134         Cats
02135         Cats
02136         Cats
02151         Cats
02163         Cats
02199         Cats
02201        Birds
02203         Cats
02210         Cats
02215         Cats
02446         Cats
02467         Cats
Name: animal, dtype: object

In [14]:
top_10_animals = df.groupby('location_zipcode')['animal'].agg(lambda x: list(x.value_counts().head(10).index))
with pd.option_context('display.max_columns', None, 'display.width', None):
    print(top_10_animals.head(n=100))

location_zipcode
00000    [Cats, Dogs, Squirells, Raccoons, Bunnies, Bir...
02108    [Cats, Dogs, Birds, Squirells, Pigeons, Swans,...
02109    [Cats, Birds, Dogs, Seagulls, Pigeons, Hawks, ...
02110    [Cats, Birds, Dogs, Seagulls, Pigeons, Gulls, ...
02111    [Cats, Birds, Seagulls, Dogs, Pigeons, Rabbits...
02113    [Cats, Birds, Pigeons, Dogs, Seagulls, Rabbits...
02114    [Cats, Dogs, Birds, Pigeons, Goose, Squirells,...
02115    [Cats, Birds, Dogs, Goose, Rabbits, Ducks, Rac...
02116    [Cats, Dogs, Birds, Squirells, Rabbits, Pigeon...
02118    [Cats, Dogs, Birds, Squirells, Seagulls, Rabbi...
02119    [Cats, Dogs, Squirells, Birds, Raccoons, Skunk...
02120    [Cats, Dogs, Birds, Raccoons, Squirells, Rabbi...
02121    [Cats, Dogs, Squirells, Raccoons, Birds, Skunk...
02122    [Cats, Dogs, Squirells, Rabbits, Birds, Raccoo...
02124    [Cats, Dogs, Squirells, Raccoons, Skunks, Poss...
02125    [Cats, Dogs, Birds, Raccoons, Squirells, Rabbi...
02126    [Cats, Dogs, Squirells, Raccoo

In [16]:
top_10_animals_df = top_10_animals.apply(pd.Series)

# Reset index to make 'zipcode' a column and adjust column names
top_10_animals_df.reset_index(inplace=True)
top_10_animals_df.columns = ['zipcode'] + [f'Rank {i}' for i in range(1, len(top_10_animals_df.columns))]

top_10_animals_df

,zipcode,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Rank 6,Rank 7,Rank 8,Rank 9,Rank 10
0,00000,Cats,Dogs,Squirells,Raccoons,Bunnies,Birds,Owl,Coyotes,NaN,NaN
1,02108,Cats,Dogs,Birds,Squirells,Pigeons,Swans,Rabbits,Goose,Hawks,Seagulls
2,02109,Cats,Birds,Dogs,Seagulls,Pigeons,Hawks,Rabbits,Mouse,Squirells,Ducks
3,02110,Cats,Birds,Dogs,Seagulls,Pigeons,Gulls,Rabbits,Turkeys,Ducks,Raccoons
4,02111,Cats,Birds,Seagulls,Dogs,Pigeons,Rabbits,Gulls,Turkeys,Skunks,Snakes
5,02113,Cats,Birds,Pigeons,Dogs,Seagulls,Rabbits,Raccoons,Owl,Mouse,Bunnies
6,02114,Cats,Dogs,Birds,Pigeons,Goose,Squirells,Seagulls,Rabbits,Hawks,Mouse
7,02115,Cats,Birds,Dogs,Goose,Rabbits,Ducks,Raccoons,Bats,Squirells,Coyotes
8,02116,Cats,Dogs,Birds,Squirells,Rabbits,Pigeons,Ducks,Seagulls,Goose,Hawks
9,02118,Cats,Dogs,Birds,Squirells,Seagulls,Rabbits,Raccoons,Pigeons,Hawks,Mouse


# Most Common Animal Per Zipcode Map

In [19]:
most_common_animals_dict = most_common_animals.to_dict()

In [23]:
import folium
from branca.element import Template, MacroElement


# GeoJSON data for Boston's zip code areas
boston_geojson = '../Datasets/ZIP_Codes.geojson'


# Mapping from animal to color
animal_to_color = {
    'Squirells': 'red', 'Cats': 'blue', 'fish': 'green',
    'Birds': 'yellow', 'dog': 'orange'
}

# Initialize the map centered around Boston
m = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

# Function to get color based on animal
# def get_color(zip_code):
#     animal = zip_to_animal.get(zip_code)
#     return animal_to_color.get(animal, 'grey')  # Use grey for zip codes not in the dictionary

def get_color(zip_code):
    animal = most_common_animals_dict.get(zip_code)
    color = animal_to_color.get(animal, 'grey')
    # print(f"Zip Code: {zip_code}, Animal: {animal}, Color: {color}")  # Debug print
    return color


# Add the GeoJSON layer with style function
folium.GeoJson(
    boston_geojson,
    name='zip code',
    style_function=lambda feature: {
        'fillColor': get_color(feature['properties'].get('ZIP5', '00000')),
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.5
    }
).add_to(m)

legend_html = '''
<div style="position: fixed; 
     bottom: 20px; left: 20px; width: 120px; height: 170px; 
     background-color: white; z-index:9999; font-size:14px;
     border:2px solid grey; border-radius:6px; padding: 10px;
     ">&nbsp;<b>Animal Legend</b><br>
     &nbsp;Cat: <i style="background:mediumslateblue; color:mediumslateblue;">&nbsp;&nbsp;</i><br>
     &nbsp;Dog: <i style="background:indianred; color:indianred;">&nbsp;&nbsp;</i><br>
     &nbsp;Rabbit: <i style="background:seashell; color:seashell;">&nbsp;&nbsp;</i><br>
     &nbsp;Bird: <i style="background:darkviolet; color:darkviolet;">&nbsp;&nbsp;</i><br>
     &nbsp;Fish: <i style="background:mediumaquamarine; color:mediumaquamarine;">&nbsp;&nbsp;</i>
</div>
'''

# # Adding the legend to the map
# legend = folium.map.Marker(
#     [42.3601, -71.0589],
#     icon=folium.DivIcon(
#         icon_size=(150,150),
#         icon_anchor=(0,0),
#         html=legend_html,
#     )
# )

# m.add_child(legend)

# Display the map
m

In [25]:
import json

def print_structure(obj, indent=0):
    """Recursive function to print the structure of a JSON object."""
    # Determine the type of the `obj` parameter
    if isinstance(obj, dict):
        for key, value in obj.items():
            # Print the current key with indentation
            print('  ' * indent + str(key))
            # Recursively print the structure of the value
            print_structure(value, indent + 1)
    elif isinstance(obj, list):
        # Print the indication of a list and its size
        print('  ' * indent + f'List - {len(obj)} items')
        # Optionally, handle lists: you can choose to print the structure of the first item, all items, or none
        if len(obj) > 0:
            # Example: print the structure of the first item
            print_structure(obj[0], indent + 1)
    else:
        # Print the type of the primitive
        print('  ' * indent + f'Value type: {type(obj).__name__}')

# Load your JSON file
filename = '../Datasets/USA_ZIP_Code_Boundaries.geojson'
with open(filename, 'r') as file:
    data = json.load(file)

# Print the structure of the loaded JSON data
print("JSON Structure:")
print_structure(data)


JSON Structure:
type
  Value type: str
name
  Value type: str
crs
  type
    Value type: str
  properties
    name
      Value type: str
features
  List - 32268 items
    type
      Value type: str
    properties
      OBJECTID
        Value type: int
      ZIP_CODE
        Value type: str
      PO_NAME
        Value type: str
      STATE
        Value type: str
      POPULATION
        Value type: NoneType
      POP_SQMI
        Value type: NoneType
      SQMI
        Value type: float
      Shape__Area
        Value type: float
      Shape__Length
        Value type: float
    geometry
      type
        Value type: str
      coordinates
        List - 6 items
          List - 1 items
            List - 65 items
              List - 2 items
                Value type: float


In [26]:
import json

def extract_zipcode_data(json_data, selected_zipcodes):
    """Extract data for specified zip codes from JSON data."""
    # Filter features by selected zip codes
    filtered_features = [
        feature for feature in json_data['features']
        if feature['properties']['ZIP_CODE'] in selected_zipcodes
    ]
    
    # Return a new JSON-like structure with filtered features
    return {
        "type": json_data['type'],
        "name": json_data['name'],
        "crs": json_data['crs'],
        "features": filtered_features
    }

# Load your JSON file
filename = '../Datasets/USA_ZIP_Code_Boundaries.geojson'
with open(filename, 'r') as file:
    data = json.load(file)

# List of zip codes you are interested in
selected_zipcodes = ["02108", "02109", "02110", "02111", "02113", "02114", "02115", "02116", "02118", "02119", "02120", "02121", "02122", "02124", "02125", "02126", "02127", "02128", "02129", "02130", "02131", "02132", "02133", "02134", "02135", "02136", "02151", "02163", "02199", "02201", "02203", "02210", "02215", "02446", "02467"]

# Extract the data for selected zip codes
filtered_data = extract_zipcode_data(data, selected_zipcodes)

# Optionally, save the filtered data to a new JSON file
filtered_filename = '../Datasets/boston_zipcodes.geojson'
with open(filtered_filename, 'w') as outfile:
    json.dump(filtered_data, outfile, indent=2)

# If you need to directly work with filtered_data in Python, you can do so as well


In [29]:
import folium
from branca.element import Template, MacroElement


# GeoJSON data for Boston's zip code areas
boston_geojson = '../Datasets/boston_zipcodes.geojson'


# Mapping from animal to color
animal_to_color = {
    'Squirells': 'red', 'Cats': 'blue', 'fish': 'green',
    'Birds': 'yellow', 'dog': 'orange'
}

# Initialize the map centered around Boston
m = folium.Map(location=[42.3601, -71.0589], zoom_start=12)

def get_color(zip_code):
    animal = most_common_animals_dict.get(zip_code)
    color = animal_to_color.get(animal, 'grey')
    print(f"Zip Code: {zip_code}, Animal: {animal}, Color: {color}")  # Debug print
    return color


# Add the GeoJSON layer with style function
folium.GeoJson(
    boston_geojson,
    name='zip code',
    style_function=lambda feature: {
        'fillColor': get_color(feature['properties'].get('ZIP_CODE', '00000')),
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.5
    }
).add_to(m)

# Display the map
m

Zip Code: 02108, Animal: Cats, Color: blue
Zip Code: 02108, Animal: Cats, Color: blue
Zip Code: 02109, Animal: Cats, Color: blue
Zip Code: 02110, Animal: Cats, Color: blue
Zip Code: 02111, Animal: Cats, Color: blue
Zip Code: 02113, Animal: Cats, Color: blue
Zip Code: 02114, Animal: Cats, Color: blue
Zip Code: 02115, Animal: Cats, Color: blue
Zip Code: 02116, Animal: Cats, Color: blue
Zip Code: 02118, Animal: Cats, Color: blue
Zip Code: 02119, Animal: Cats, Color: blue
Zip Code: 02120, Animal: Cats, Color: blue
Zip Code: 02121, Animal: Cats, Color: blue
Zip Code: 02122, Animal: Cats, Color: blue
Zip Code: 02124, Animal: Cats, Color: blue
Zip Code: 02125, Animal: Cats, Color: blue
Zip Code: 02126, Animal: Cats, Color: blue
Zip Code: 02127, Animal: Cats, Color: blue
Zip Code: 02128, Animal: Cats, Color: blue
Zip Code: 02129, Animal: Cats, Color: blue
Zip Code: 02130, Animal: Cats, Color: blue
Zip Code: 02131, Animal: Cats, Color: blue
Zip Code: 02132, Animal: Cats, Color: blue
Zip Code: 0